## Atividades - Resolver com DataFrames

## Crie uma consulta que mostre, nesta ordem, Nome, Estados e Status.

In [97]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import functions as Func

In [98]:
!hdfs dfs -ls /user/manoel/data/download/Atividades

Found 5 items
-rw-r--r--   3 root supergroup       9311 2023-01-16 22:54 /user/manoel/data/download/Atividades/Clientes.parquet
-rw-r--r--   3 root supergroup      21142 2023-01-16 22:54 /user/manoel/data/download/Atividades/ItensVendas.parquet
-rw-r--r--   3 root supergroup       3490 2023-01-16 22:54 /user/manoel/data/download/Atividades/Produtos.parquet
-rw-r--r--   3 root supergroup      11882 2023-01-16 22:54 /user/manoel/data/download/Atividades/Vendas.parquet
-rw-r--r--   3 root supergroup       2589 2023-01-16 22:54 /user/manoel/data/download/Atividades/Vendedores.parquet


In [108]:
clientes = spark.read.load("/user/manoel/data/download/Atividades/Clientes.parquet")

In [114]:
dados_cliente = clientes.select("Cliente","Estado","Status")

In [115]:
dados_cliente.show()

+--------------------+------+--------+
|             Cliente|Estado|  Status|
+--------------------+------+--------+
|Adelina Buenaventura|    RJ|  Silver|
|        Adelino Gago|    RJ|  Silver|
|     Adolfo Patrício|    PE|  Silver|
|    Adriana Guedelha|    RO|Platinum|
|       Adélio Lisboa|    SE|  Silver|
|       Adérito Bahía|    MA|  Silver|
|       Aida Dorneles|    RN|  Silver|
|   Alarico Quinterno|    AC|  Silver|
|    Alberto Cezimbra|    AM|  Silver|
|    Alberto Monsanto|    RN|    Gold|
|       Albino Canela|    AC|  Silver|
|     Alceste Varanda|    RR|  Silver|
|  Alcides Carvalhais|    RO|  Silver|
|        Aldo Martins|    GO|  Silver|
|   Alexandra Tabares|    MG|  Silver|
|      Alfredo Cotrim|    SC|  Silver|
|     Almeno Figueira|    SC|  Silver|
|      Alvito Peralta|    AM|  Silver|
|     Amadeu Martinho|    RN|  Silver|
|      Amélia Estévez|    PE|  Silver|
+--------------------+------+--------+
only showing top 20 rows



## Crie uma consulta que mostre apenas os clientes do Status "Platinum" e "Gold".

In [121]:
stastus_cli = clientes.select("Cliente", "Estado", "Status").where((Func.col("Status") == "Platinum") | (Func.col("Status") == "Gold")).orderBy("Status")

In [122]:
stastus_cli.show()

+-------------------+------+--------+
|            Cliente|Estado|  Status|
+-------------------+------+--------+
|      Carminda Dias|    AM|    Gold|
|   Alberto Monsanto|    RN|    Gold|
|   Firmino Meireles|    AM|    Gold|
|Honorina Villaverde|    PE|    Gold|
|  Iracema Rodríguez|    BA|    Gold|
|      Cláudio Jorge|    TO|    Gold|
|      Anna Carvajal|    RS|    Gold|
|    Dionísio Saltão|    PR|    Gold|
|      Bento Quintão|    SP|    Gold|
|   Adriana Guedelha|    RO|Platinum|
|      Flor Vilanova|    CE|Platinum|
|    Ibijara Botelho|    RR|Platinum|
|         Joana Ataí|    GO|Platinum|
+-------------------+------+--------+



## Demostre quanto cada Status de Clientes representa em vendas?

In [111]:
Vendas = spark.read.load("/user/manoel/data/download/Atividades/Vendas.parquet")

In [118]:
Vendas.show()

+--------+----------+---------+---------+--------+
|VendasID|VendedorID|ClienteID|     Data|   Total|
+--------+----------+---------+---------+--------+
|       1|         1|       91| 1/1/2019|  8053.6|
|       2|         6|      185| 1/1/2020|   150.4|
|       3|         7|       31| 2/1/2020|  6087.0|
|       4|         5|       31| 2/1/2019| 13828.6|
|       5|         5|       31| 3/1/2018|26096.66|
|       6|         5|       31| 4/1/2020| 18402.0|
|       7|         5|       31| 6/1/2019|  7524.2|
|       8|         5|      186| 6/1/2019| 12036.6|
|       9|         7|       91| 6/1/2020| 2804.75|
|      10|         2|      202| 6/1/2020|  8852.0|
|      11|         7|       58| 8/1/2019|16545.25|
|      12|         7|       58| 9/1/2018|11411.88|
|      13|         7|       58|10/1/2019| 15829.7|
|      14|         3|      249|12/1/2020| 6154.36|
|      15|         4|      249|12/1/2018| 3255.08|
|      16|         7|      192|13/1/2020| 2901.25|
|      17|         2|       79|

In [143]:
join_clientes_vendas = clientes.join(Vendas, clientes.ClienteID == Vendas.ClienteID, 'inner').select(clientes.ClienteID, clientes.Status, Vendas.Total).groupBy(clientes.Status).agg(sum("Total")).orderBy(Func.col("sum(Total)").desc())

In [144]:
join_clientes_vendas.show()

+--------+------------------+
|  Status|        sum(Total)|
+--------+------------------+
|  Silver|        3014291.36|
|    Gold|27286.690000000002|
|Platinum|          12584.68|
+--------+------------------+

